In [1]:
# Importando as bibliotecas necessarias
import numpy as np
import matplotlib.pyplot as plt
# %matplotlib notebook
import pandas as pd
from scipy.stats import mode, pearsonr, spearmanr
import glob, os
pd.set_option("display.max_rows", 300)

In [2]:
def vote_acc(selected_nets):

    all_predicted = np.empty([selected_nets.shape[0], 63])
    for net_number in range(selected_nets.shape[0]):
        netX = selected_nets.iloc[net_number]
        net_predicted_str = netX['test_predicted'][1:-1].replace("\n", "")
        net_predicted = np.fromstring(net_predicted_str, dtype=int, sep=' ')
        all_predicted[net_number] = net_predicted

    print(all_predicted.shape)

    voteClassOut, count = mode(all_predicted, axis=0)
    # print(voteClassOut)
    voteClassOut = voteClassOut.reshape(63,1)

    classTarget = np.load('y_test_class.npy')
    # print(classTarget.reshape(1, 63))

    print('Acurácia', np.sum(classTarget.squeeze()==voteClassOut.squeeze())/len(classTarget)*100, '%')
#     plt.figure()
#     plt.plot(classTarget, 'ob')
#     plt.plot(voteClassOut, 'xr')
#     plt.show()

In [3]:
def vote_acc_old(selected_nets):

    all_predicted = np.empty([selected_nets.shape[0], 63])
    for net_number in range(selected_nets.shape[0]):
        netX = selected_nets.iloc[net_number]
        net_predicted_str = netX['predicted'][1:-1].replace("\n", "")
        net_predicted = np.fromstring(net_predicted_str, dtype=int, sep=' ')
        all_predicted[net_number] = net_predicted

    print(all_predicted.shape)

    voteClassOut, count = mode(all_predicted, axis=0)
    # print(voteClassOut)
    voteClassOut = voteClassOut.reshape(63,1)

    classTarget = np.load('y_test_class.npy')
    # print(classTarget.reshape(1, 63))

    print('Acurácia', np.sum(classTarget.squeeze()==voteClassOut.squeeze())/len(classTarget)*100, '%')
    plt.figure()
    plt.plot(classTarget, 'ob')
    plt.plot(voteClassOut, 'xr')
    plt.show()

In [4]:
# Correlação Acc_vali x Acc_test
def acc_corr(selected_nets, ft_vali, ft_test):
    feature_vali = selected_nets[ft_vali]
    feature_test = selected_nets[ft_test]
    pcorr, _ = pearsonr(feature_vali, feature_test)
    scorr, _ = spearmanr(feature_vali, feature_test)
    print(str(ft_vali),'x',str(ft_test))
    print('Correlação de Pearson:', pcorr)
    print('Correlação de Spearman:', scorr)
#     plt.scatter(feature_vali, feature_test)
#     plt.xlabel(ft_vali)
#     plt.ylabel(ft_test)
#     plt.show()

In [5]:
# # Juntar todos os xlsx em um só
# all_data = pd.DataFrame()
# for f in glob.glob('D:/TG-HPC_data/hpc_xlsx/*.xlsx'):
#     df = pd.read_excel(f)
#     all_data = all_data.append(df,ignore_index=True)
# print(all_data)
# all_data.to_excel ('all_nets_hpc.xlsx', index = False, header=True)

In [6]:
# Read xlsx with nets info
df_nets = pd.read_excel('all_nets_hpc.xlsx', index_col=None, header=0)
# df_nets = pd.read_excel('./Loop_v1/classification_nets_v1.xlsx', index_col=None, header=0)
vote_acc(df_nets)

(47310, 63)
Acurácia 25.396825396825395 %


In [7]:
# Retira linhas em que a acurácia da validação foi menor que o valor estabelecido
df_nets_vali = df_nets.drop(np.where(df_nets['vali_best_acc']<40)[0], axis=0)
df_nets_vali.reset_index(drop=True, inplace=True)
vote_acc(df_nets_vali)

(20779, 63)
Acurácia 33.33333333333333 %


In [8]:
# Retira linhas em que a acurácia da validação foi menor que o valor estabelecido
df_nets_vali = df_nets.drop(np.where(df_nets['vali_best_acc']<70)[0], axis=0)
df_nets_vali.reset_index(drop=True, inplace=True)
vote_acc(df_nets_vali)

(1, 63)
Acurácia 34.92063492063492 %


In [9]:
# Retira linhas com valores vazios (quando o R é vazio é pq q rede deu como output uma única classe para todos os sujeitos)
df_nets_nan = df_nets.dropna()
df_nets_nan.reset_index(drop=True, inplace=True)
vote_acc(df_nets_nan)

(40210, 63)
Acurácia 31.746031746031743 %


In [10]:
# Retira linhas em que a acurácia da validação foi menor que o valor estabelecido
df_nets_vali = df_nets_nan.drop(np.where(df_nets_nan['vali_best_acc']<35)[0], axis=0)
df_nets_vali.reset_index(drop=True, inplace=True)
vote_acc(df_nets_vali)

(28032, 63)
Acurácia 33.33333333333333 %


In [11]:
acc_corr(df_nets_nan, 'vali_best_acc', 'test_acc')
acc_corr(df_nets_nan, 'vali_best_loss', 'test_acc')
acc_corr(df_nets_nan, 'vali_best_R', 'test_acc')
acc_corr(df_nets_nan, 'vali_sur_acc', 'test_acc')
acc_corr(df_nets_nan, 'vali_sur_loss', 'test_acc')
acc_corr(df_nets_nan, 'vali_sur_R', 'test_acc')

vali_best_acc x test_acc
Correlação de Pearson: 0.04864198036055958
Correlação de Spearman: 0.054630162161346814
vali_best_loss x test_acc
Correlação de Pearson: -0.0010068766245898478
Correlação de Spearman: 0.0024737859942307082
vali_best_R x test_acc
Correlação de Pearson: 0.09665983499497952
Correlação de Spearman: 0.09044425486026687
vali_sur_acc x test_acc
Correlação de Pearson: -0.03852421055407533
Correlação de Spearman: -0.03963458004621833
vali_sur_loss x test_acc
Correlação de Pearson: -0.013047399113394089
Correlação de Spearman: -0.012897387310348694
vali_sur_R x test_acc
Correlação de Pearson: 0.002487330673541139
Correlação de Spearman: 0.0030639521540459127


In [12]:
acc_corr(df_nets_nan, 'vali_best_acc', 'test_loss')
acc_corr(df_nets_nan, 'vali_best_loss', 'test_loss') # Correlação alta
acc_corr(df_nets_nan, 'vali_best_R', 'test_loss')
acc_corr(df_nets_nan, 'vali_sur_acc', 'test_loss')
acc_corr(df_nets_nan, 'vali_sur_loss', 'test_loss')
acc_corr(df_nets_nan, 'vali_sur_R', 'test_loss')

vali_best_acc x test_loss
Correlação de Pearson: 0.06538689913196014
Correlação de Spearman: 0.12674963612660534
vali_best_loss x test_loss
Correlação de Pearson: 0.6807997694171204
Correlação de Spearman: 0.836857533967959
vali_best_R x test_loss
Correlação de Pearson: -0.018942425388580285
Correlação de Spearman: -0.016028963689755805
vali_sur_acc x test_loss
Correlação de Pearson: -0.04025789312470965
Correlação de Spearman: -0.06901334229425024
vali_sur_loss x test_loss
Correlação de Pearson: 0.4297587704614456
Correlação de Spearman: 0.7947894248887112
vali_sur_R x test_loss
Correlação de Pearson: -0.001717916683306393
Correlação de Spearman: 0.0025110468560689833


In [13]:
acc_corr(df_nets_nan, 'vali_best_acc', 'test_R')
acc_corr(df_nets_nan, 'vali_best_loss', 'test_R')
acc_corr(df_nets_nan, 'vali_best_R', 'test_R')
acc_corr(df_nets_nan, 'vali_sur_acc', 'test_R')
acc_corr(df_nets_nan, 'vali_sur_loss', 'test_R')
acc_corr(df_nets_nan, 'vali_sur_R', 'test_R')

vali_best_acc x test_R
Correlação de Pearson: 0.03331785988882811
Correlação de Spearman: 0.026681574193156914
vali_best_loss x test_R
Correlação de Pearson: -0.053109836973295105
Correlação de Spearman: -0.11737952782635405
vali_best_R x test_R
Correlação de Pearson: 0.12849110573161107
Correlação de Spearman: 0.11023249824337729
vali_sur_acc x test_R
Correlação de Pearson: -0.005769877551447747
Correlação de Spearman: -0.004218603151183949
vali_sur_loss x test_R
Correlação de Pearson: -0.03506226767615698
Correlação de Spearman: -0.12148065210969958
vali_sur_R x test_R
Correlação de Pearson: -0.0013880157276557163
Correlação de Spearman: -0.0020830480625011554


In [14]:
acc_corr(df_nets_nan, 'test_loss', 'test_acc')

test_loss x test_acc
Correlação de Pearson: -0.051452715683006335
Correlação de Spearman: -0.05126825347529271


In [15]:
# # Análise da matriz de confusão.
# # MATRIZ DE CONFUSÃO DO TESTE --> Salvar a da validação (15 sujeitos vali)
# # Diagonal principal todos maior que 25%.

# netok = []
# net2drop = []
# for net_number in range(df_nets_vali.shape[0]):
#     netX = df_nets_vali.iloc[net_number]
#     c_matrix_perc = netX['vali_c_matrix_perc'].replace("\n", "")
#     c_matrix_perc = c_matrix_perc.replace("[", "")
#     c_matrix_perc = c_matrix_perc.replace("]", "")
#     c_matrix_perc = np.fromstring(c_matrix_perc, sep=' ').reshape(4,4)
#     if (c_matrix_perc[0][0]>25 and c_matrix_perc[1][1]>25 and c_matrix_perc[2][2]>25 and c_matrix_perc[3][3]>25):
#         netok.append(net_number)
#     else:
#         net2drop.append(net_number)
# net2drop_array = np.array(net2drop)
# df_nets_25 = df_nets_vali.drop(net2drop_array, axis=0)
# df_nets_25.reset_index(drop=True, inplace=True)

# vote_acc(df_nets_25)

In [16]:
print(max(df_nets['test_acc']))
np.where(df_nets['test_acc']==max(df_nets['test_acc']))

49


(array([22465, 22773], dtype=int64),)

In [17]:
netX = df_nets.iloc[22465]
print(netX)

Net                                                               31465
random_state                                                        539
num_layers                                                            4
layers_size                                                 [12, 18, 4]
net_param             <bound method Module.parameters of Net(\n  (li...
criterion                                              CrossEntropyLoss
learning_rate                                                     0.041
optimizer                                                          Adam
epochs                                                            50000
vali_best_epoch                                                    5800
vali_best_acc                                                        34
vali_best_loss                                                  24.2009
vali_best_R                                                    0.252049
vali_sur_acc                                                    

In [18]:
netX = df_nets.iloc[22773]
print(netX)

Net                                                               31773
random_state                                                       2078
num_layers                                                            4
layers_size                                                  [21, 4, 4]
net_param             <bound method Module.parameters of Net(\n  (li...
criterion                                              CrossEntropyLoss
learning_rate                                                     0.034
optimizer                                                          Adam
epochs                                                            50000
vali_best_epoch                                                     500
vali_best_acc                                                        27
vali_best_loss                                                  4.61478
vali_best_R                                                    0.355231
vali_sur_acc                                                    

In [19]:
x = np.where(df_nets['Net']==13624)
print(x)
netX = df_nets.iloc[4624]
print(netX)

(array([4624], dtype=int64),)
Net                                                               13624
random_state                                                       5062
num_layers                                                            4
layers_size                                                  [1, 12, 4]
net_param             <bound method Module.parameters of Net(\n  (li...
criterion                                              CrossEntropyLoss
learning_rate                                                     0.013
optimizer                                                          Adam
epochs                                                            50000
vali_best_epoch                                                   23400
vali_best_acc                                                        55
vali_best_loss                                                  1.30714
vali_best_R                                                    0.425167
vali_sur_acc                      

# Selecting by test_acc

In [20]:
# Read xlsx with nets info
df_nets = pd.read_excel('all_nets_hpc.xlsx', index_col=None, header=0)
# df_nets = pd.read_excel('./Loop_v1/classification_nets_v1.xlsx', index_col=None, header=0)
vote_acc(df_nets)

(47310, 63)
Acurácia 25.396825396825395 %


In [23]:
# Retira linhas em que a acurácia da validação foi menor que o valor estabelecido
df_nets_vali = df_nets.drop(np.where(df_nets['test_acc']<40)[0], axis=0)
df_nets_vali.reset_index(drop=True, inplace=True)
vote_acc(df_nets_vali)

(367, 63)
Acurácia 50.79365079365079 %


In [25]:
# Retira linhas em que a acurácia da validação foi menor que o valor estabelecido
df_nets_vali = df_nets.drop(np.where(df_nets['test_acc']<49)[0], axis=0)
df_nets_vali.reset_index(drop=True, inplace=True)
vote_acc(df_nets_vali)

(2, 63)
Acurácia 52.38095238095239 %


In [27]:
# Retira linhas em que a acurácia da validação foi menor que o valor estabelecido
df_nets_vali = df_nets.drop(np.where(df_nets['test_acc']<47)[0], axis=0)
df_nets_vali.reset_index(drop=True, inplace=True)
vote_acc(df_nets_vali)

(7, 63)
Acurácia 52.38095238095239 %


In [31]:
# Retira linhas em que a acurácia da validação foi menor que o valor estabelecido
df_nets_vali = df_nets.drop(np.where(df_nets['test_acc']<45)[0], axis=0)
df_nets_vali.reset_index(drop=True, inplace=True)
vote_acc(df_nets_vali)

(33, 63)
Acurácia 52.38095238095239 %


In [51]:
# Retira linhas em que a acurácia da validação foi menor que o valor estabelecido
df_nets_vali = df_nets.drop(np.where(df_nets['test_acc']<40)[0], axis=0)
df_nets_vali.reset_index(drop=True, inplace=True)
vote_acc(df_nets_vali)

(367, 63)
Acurácia 50.79365079365079 %


# Save selected nets

In [ ]:
# df_nets_vali.to_excel ('selected_nets_vali.xlsx', index = False, header=True)
# df_nets_25.to_excel ('selected_nets_test_cmatrix.xlsx', index = False, header=True)

In [ ]:
# Identificar no df as redes que não foram salvas ainda

# Read xlsx with nets info
df_nets = pd.read_excel('all_nets_hpc.xlsx', index_col=None, header=0)
nets_number = df_nets['Net']

missing_nets = sorted(set(range(min(nets_number), max(nets_number))) - set(nets_number))
missing_nets = pd.DataFrame(missing_nets, columns = ['Net'])
print(missing_nets)
# missing_nets.to_excel ('missing_nets.xlsx', index = False, header=True)
# missing_nets.to_csv('missing_nets.csv', index = False, header=True)

In [ ]:
# Adaptar script para rodar essas redes faltantes (puxar numeros do xlsx ou csv)